# Preprocessing

In [ ]:
import os

try:
  if not os.path.isdir('pa_county'):
    !unzip pa_County.zip -d pa_county
  pass
except:
  pass

try:
  import geopandas as gpd
except:
  !pip install geopandas
  import geopandas as gpd

from matplotlib import pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

fp = "pa_county/PaCounty2024_03.shp"

#reading the file stored in variable fp
map_df = gpd.read_file(fp)
map_df = map_df.to_crs(epsg=4326)
map_df.plot()

In [ ]:
#sort according to fips
#map_df = map_df.sort_values('GEOID')

def swap_columns(df, col1, col2):
    col_list = list(df.columns)
    x, y = col_list.index(col1), col_list.index(col2)
    col_list[y], col_list[x] = col_list[x], col_list[y]
    df = df[col_list]
    return df

#to make Allegheny county the first row as it's in the data
def swap_rows(df, i1, i2):
    a, b = df.iloc[i1, :].copy(), df.iloc[i2, :].copy()
    df.iloc[i1, :], df.iloc[i2, :] = b, a
    return df

map_df = map_df.sort_values('FIPS_COUNT')
map_df = map_df.reset_index(drop=True)
#map_df = swap_columns(map_df, 'FIPS_COUNT', 'GEOID')

# 1 in case we're using oud_pa2.csv
oud_file = 1
if oud_file == 1:
  map_df = swap_rows(map_df, 0, 1)
map_df = map_df.reset_index(drop=True)
map_df = map_df.rename(columns={'FIPS_COUNT': 'FIPS'})
map_df = map_df.rename(columns={'COUNTY_NAM': 'NAME'})
map_df.head()

In [ ]:
import pandas as pd
import numpy as np
import torch

df = [0]*25
print(df)

for i in range(5):
  for j in range(5):
    df_temp = pd.read_csv('data_pa_int_'+str(i+1)+str(j+1)+'.csv')
    df_temp = df_temp.dropna()
    df_temp.columns = map_df['NAME'].tolist()
    df[i*5+j] = df_temp

In [ ]:
df_pop = pd.read_csv('pop_pa.csv')
df_pop = df_pop.dropna()
df_pop = df_pop.drop_duplicates(subset='County Name', keep="last")
delete_row = df_pop[df_pop['County Name'] == 'Pennsylvania'].index
df_pop = df_pop.drop(delete_row)
df_pop = df_pop.sort_values('County Name')
df_pop = swap_rows(df_pop, 0, 1)
df_pop = df_pop.reset_index(drop=True)

area = map_df['AREA_SQ_MI'].tolist()
pop = df_pop['Population'].tolist()
pop_den = pop/np.array(area)

In [ ]:
import re

df_income = pd.read_csv('house_income.csv')
df_income = df_income.dropna()

df_income = swap_rows(df_income, 0, 1)
df_income = df_income.reset_index(drop=True)

income = df_income['Income'].tolist()
income = [int(i/1000) for i in income]

df_black_percent = pd.read_csv('pa_black_percent.csv')
df_black_percent = df_black_percent.dropna()

black_p = df_black_percent['Percent'].tolist()

df_rurality = pd.read_csv('rurality.csv')
df_rurality = df_rurality.dropna()

rurality = df_rurality['Index'].tolist()

df_unemploy = pd.read_csv('unemploy_pa.csv')
df_unemploy = df_unemploy.dropna()
unemploy = df_unemploy['rate'].tolist()

In [ ]:
map_df['FIPS'] = ('42'+map_df['FIPS'])

### Indicate which files to use i.e., counts or rates

In [ ]:
'''
change_file = 0
if change_file == 1:
  df = df_abs
  df_int2 = df_abs_int2
  df_int3 = df_abs_int3
  df_int4 = df_abs_int4
  df_int5 = df_abs_int5

  df_average = [0]*len(map_df)
  df_std = [0]*len(map_df)
  for i in range(len(df.iloc[0])-2):
    ls = []
    for j in range(len(df)-1):
      #print(i,j,df.iloc[j][i])
      ls.append(int(df.iloc[j][i]))

    df_average[i] = sum(ls) / len(ls)
    df_std[i] = np.std(ls)

'''
change_file = 0
if change_file == 1:
  for i in range(5):
    for j in range(5):
      df[i*5+j] =  np.log(df[i*5+j]/np.array(pop)*100000 + 1)

In [ ]:

import numpy as np
arr = [float(df[0][map_df['NAME'].loc[i]].loc[i]) for i in range(len(map_df))]
merged = map_df.join(pd.DataFrame(arr,columns=['ODD']))

merged.head()



# GPR related functions
### Loading required libraries and creating essential functions

In [ ]:
import torch
import numpy as np
try:
  from botorch.models import SingleTaskGP, Ex, MultiTaskGP
except:
  !pip install botorch
  from botorch.models import SingleTaskGP, MultiTaskGP

from botorch.fit import fit_gpytorch_mll
from gpytorch.mlls import ExactMarginalLogLikelihood, MarginalLogLikelihood
from botorch.models.transforms.outcome import Standardize

from gpytorch.kernels import MaternKernel, RBFKernel, LinearKernel, RQKernel, SpectralDeltaKernel, SpectralMixtureKernel, ScaleKernel, IndexKernel, PeriodicKernel


from scipy.stats import qmc
sampler = qmc.LatinHypercube(d=2)
sample = sampler.random(n=1)

batch_size = 1

ls = np.array([map_df['geometry'].loc[0].centroid.x, map_df['geometry'].loc[0].centroid.y,
               #income[0],
               pop_den[0], #black_p[0],
               #native_p[0], #poverty[0]
               ]).reshape(1,-1)
y = np.array(float(df[0][map_df['NAME'].iloc[0]].iloc[0])).reshape(-1,1)

for i in range(1,len(map_df)):
  #print(i)
  for j in range(batch_size):
    ls = np.append(ls, np.array([map_df['geometry'].loc[i].centroid.x, map_df['geometry'].loc[i].centroid.y,
                                 #income[i],
                                 pop_den[i], #black_p[i], #native_p[i]
                                 #native_p[i], #poverty[i]
                                 ]).reshape(1,-1), axis=0)
    y = np.append(y, np.array(float(df[0][map_df['NAME'].iloc[i]].iloc[j+1])).reshape(-1,1), axis=0)

#train_X = torch.rand(10, 2, dtype=torch.float64)
train_X = torch.Tensor(np.array(ls)).double()
train_Y = torch.Tensor(y, ).double()

part = [ls[:,i] for i in range(np.shape(ls)[1])]

choice = torch.Tensor(np.array([i for i in part]))
#print(choice, choice.shape)

In [ ]:
from botorch.acquisition import UpperConfidenceBound
from botorch.acquisition.monte_carlo import qExpectedImprovement, qNoisyExpectedImprovement
from botorch.acquisition.logei import qLogExpectedImprovement
from botorch.acquisition.active_learning import qNegIntegratedPosteriorVariance
from botorch.acquisition.analytic import PosteriorStandardDeviation

from botorch.acquisition.max_value_entropy_search import qMaxValueEntropy
from botorch.acquisition.predictive_entropy_search import qPredictiveEntropySearch

from botorch.optim import optimize_acqf, optimize_acqf_mixed, optimize_acqf_discrete_local_search, optimize_acqf_discrete
from botorch.sampling.normal import SobolQMCNormalSampler

try:
  import pointpats
except:
  !pip install pointpats
  import pointpats
#pointpats.random.poisson(get_all, size=1).tolist()

MC_SAMPLES = 128

# x = np.array([-80.5,39.5,0])
# y = np.array([-75.5,42,1])
# bounds = torch.stack([torch.Tensor(x), torch.Tensor(y)])
#bounds = torch.tensor([[0.0] * 6, [1.0] * 6])

In [ ]:
from math import e
import numpy as np
import random
from shapely.ops import unary_union
from shapely.geometry import Point
#from shapely.geometry import MultiPoint
from shapely.geometry.polygon import Polygon

try:
  import pointpats
except:
  !pip install pointpats
  import pointpats

from scipy.stats import qmc
sampler = qmc.LatinHypercube(d=2)
sample = sampler.random(n=10)


l_bounds = [-79.21049943375089, 40.81555532075]
u_bounds = [-78.70396386132802, 41.355336251154925]
sample_scaled = qmc.scale(sample, l_bounds, u_bounds)

x=map_df['geometry'].loc[0]
bounds = map_df.geometry.apply(lambda x: x.bounds).tolist()
point = Point(-80.07562,40.39311111)

arr = df[0].to_numpy()
arr_loc = np.zeros((len(map_df)))


def get_index(arr,df):
  for i in range(len(df.index)):
    x=map_df['geometry'].loc[i]
    point = Point(arr[0][0],arr[0][1])
    if x.contains(point): #or x.touches(point):
        return i

def get_index_relative(arr,df):
  loc=0
  for i in range(len(df.index)):
    x=map_df['geometry'].loc[i]
    point = Point(arr[0][0],arr[0][1])
    if x.contains(point): #or x.touches(point):
        loc = i
        break
  index_ls = []
  for i in range((len(df.index))):
    if x.touches(df['geometry'].loc[i]):
      index_ls.append(i)

  pop_val = [pop[i] for i in index_ls]
  return index_ls[np.argmin(pop_val)]

def adjust_loc(arr, map_df):
  e = 0.35
  arr2 = [[arr[0][0],arr[0][1]]]
  f1 = f2 = f3 = f4 = 0
  if arr2[0][0] <= -80.48:
    f1 = 1
  if arr2[0][0] >= -75.28:
    f2 = 1
  if arr2[0][1] <= 39.72:
    f3 = 1
  if arr2[0][1] >= 48.9:
    #was 42.25
    f4 = 1

  #else:
    #arr[0][0] = arr[0][0] + e
  if f1 == 1:
    arr2[0][0] = arr2[0][0] + np.abs(-1*arr2[0][0]-80.48)  + e
  if f2 == 1:
    arr2[0][0] = arr2[0][0] - np.abs(-1*arr2[0][0]-75.28)  - e
  if f3 == 1:
    arr2[0][1] = arr2[0][1] - np.abs(arr2[0][1]-39.72)  + e
  if f4 == 1:
    arr2[0][1] = arr2[0][1] - np.abs(arr2[0][1]-42.2)  - e

  if torch.is_tensor(arr2[0][0]) or torch.is_tensor(arr2[0][1]):
    arr2[0][0] = arr2[0][0].detach().tolist()
    arr2[0][1] = arr2[0][1].detach().tolist()
  arr_ls = [arr2[0][0], arr2[0][1]]
  for i in range(2,len(arr[0])):
    #print("here:",arr,i)
    if torch.is_tensor(arr[0][i]):
      arr_ls.append(arr[0][i].detach().tolist())
    else:
      arr_ls.append(arr[0][i])

  return [arr_ls]

def get_random_point(arr, map_df):
  get_all = unary_union( map_df['geometry'].tolist() )
  arr_ls = pointpats.random.poisson(get_all, size=1).tolist()
  for i in range(2,len(arr[0])):
    #print("here:",arr,i)
    if torch.is_tensor(arr[0][i]):
      arr_ls.append(arr[0][i].detach().tolist())
    else:
      arr_ls.append(arr[0][i])

  return [arr_ls]

def random_points_in_polygon(number, polygon):
    points = []
    ls = []
    min_x, min_y, max_x, max_y = polygon.bounds
    i= 0
    while i < number:
        x1 = random.uniform(min_x, max_x)
        y1 = random.uniform(min_y, max_y)
        point = Point(x1, y1)
        ls_new = [x1,y1]
        if polygon.contains(point):
            points.append(point)
            ls.append(ls_new)
            i += 1
    return ls

import geopandas as gpd

def get_neighboring_counties(state_name, map_df, county_name):
    """
    Function to return the neighboring counties of a given county in the same state.

    Args:
        state_name (str): Name of the state (e.g., "North Dakota").
        county_name (str): Name of the county (e.g., "Steele").

    Returns:
        List of neighboring county names.
    """
    # Load the U.S. counties shapefile (change the file path to your shapefile location)
    counties = gpd.read_file("https://www2.census.gov/geo/tiger/GENZ2022/shp/cb_2022_us_county_5m.zip")

    # Filter by the specific state
    state_counties = counties[counties['STATE_NAME'].str.lower() == state_name.lower()]

    # Find the target county
    target_county = state_counties[state_counties['NAME'].str.lower() == county_name.lower()]

    if target_county.empty:
        print(f"County '{county_name}' not found in '{state_name}'.")
        return []

    # Find neighboring counties using spatial intersection (buffer to account for boundary precision)
    neighbors = state_counties[state_counties.geometry.touches(target_county.geometry.iloc[0])]
    neighbors = neighbors['NAME'].tolist()
    print(neighbors)
    # Return the names of neighboring counties
    names = map_df['NAME'].tolist()
    county_indexes = [names.index(i) for i in neighbors]
    sample_size = [np.shape(train_gp_LR1[i])[0] for i in (county_indexes)]
    #neighbor_names = neighbors['NAME'].tolist()
    return np.argmin(sample_size)


def random_points_in_polygon_2(number, polygon):
  sampler = qmc.LatinHypercube(d=2)
  sample = sampler.random(n=10)
  points = []
  ls = []
  min_x, min_y, max_x, max_y = polygon.bounds
  i= 0
  l_bounds = [min_x, min_y]
  u_bounds = [max_x, max_y]

  while i < number:
        sample = sampler.random(n=number)
        ls_pnt = qmc.scale(sample, l_bounds, u_bounds)
        x1 = ls_pnt[0][0]
        y1 = ls_pnt[0][1]
        point = Point(x1, y1)
        ls_new = [x1,y1]
        if polygon.contains(point):
            points.append(point)
            ls.append(ls_new)
            i += 1
  return ls

from functools import reduce

def get_random(arr):
  sampler = qmc.LatinHypercube(d=2)
  sample = sampler.random(n=10)
  points = []
  ls = []
  min_x, min_y, max_x, max_y = arr
  i= 0
  l_bounds = [min_x, min_y]
  u_bounds = [max_x, max_y]

  while 1:
    sample = sampler.random(n=1)
    ls_pnt = qmc.scale(sample, l_bounds, u_bounds)
    x1 = ls_pnt[0][0]
    y1 = ls_pnt[0][1]
    ls_new = [x1,y1]
    point = Point(x1, y1)
    ls = map_df.contains(point).to_list()
    val =reduce(lambda x, y: x+y, ls)
    if val:
        break
  return ls_new

random_points_in_polygon_2(1,map_df['geometry'].iloc[32])

In [ ]:
if np.shape(ls)[1] > 2:
  newls = np.zeros((len(map_df),np.shape(ls)[1]))
else:
  newls = np.zeros((len(map_df),2))
for i in range(len(map_df['geometry'])):
  newls[i,0]=map_df['geometry'].loc[i].centroid.x
  newls[i,1]=map_df['geometry'].loc[i].centroid.y
  if np.shape(ls)[1] > 2:
    for j in range(2,np.shape(ls)[1]):
      #newls[i,j]=feature[j-2][i]
      #newls[i,2]=income[i]
      newls[i,2]=pop_den[i]
      #newls[i,4]=black_p[i]
      #newls[i,5]=native_p[i]
      #newls[i,4]=black_percent[i]

In [ ]:
ls_nam = map_df['NAME'].to_list()
#print(ls)

In [ ]:
import math
from typing import Optional

import gpytorch
from botorch.posteriors.gpytorch import scalarize_posterior_gpytorch
from botorch.acquisition.objective import ScalarizedPosteriorTransform
from botorch.acquisition.objective import PosteriorTransform
from botorch.acquisition.analytic import ExpectedImprovement
from botorch.acquisition.max_value_entropy_search import qMaxValueEntropy
from botorch.acquisition.monte_carlo import qExpectedImprovement

from botorch.models import HigherOrderGP, SingleTaskGP

from botorch.optim import optimize_acqf_discrete_local_search
from botorch.acquisition.monte_carlo import MCAcquisitionFunction
from botorch.models.model import Model
from botorch.sampling.base import MCSampler
from botorch.sampling.normal import SobolQMCNormalSampler
from botorch.utils import t_batch_mode_transform
from botorch.acquisition.objective import PosteriorTransform
from botorch.models import MultiTaskGP
from gpytorch.distributions import MultivariateNormal

#from botorch.models import FixedNoiseGP, MultiTaskGP

from torch import Tensor

from botorch.acquisition import AnalyticAcquisitionFunction


class MultiOutputGPModel(MultiTaskGP):
    def __init__(self, train_X, train_Y):
        #super().__init__(train_X, train_Y, output_tasks=train_Y.shape[1], task_feature=1 )
        super().__init__(train_X, train_Y, num_outputs=train_Y.shape[1])

        # Ensure we're using an appropriate likelihood for multi-output regression
        self.likelihood = self._get_likelihood()

    def _get_likelihood(self):
        return super().likelihood


class Entropy(AnalyticAcquisitionFunction):
    def __init__(
        self,
        model: Model,
        posterior_transform: Optional[PosteriorTransform] = None,
        maximize: bool = True,
    ) -> None:
        r"""Single-outcome Posterior Mean.

        Args:
            model: A fitted single-outcome GP model (must be in batch mode if
                candidate sets X will be)
            posterior_transform: A PosteriorTransform. If using a multi-output model,
                a PosteriorTransform that transforms the multi-output posterior into a
                single-output posterior is required.
            maximize: If True, consider the problem a maximization problem. Note
                that if `maximize=False`, the posterior standard deviation is negated.
                As a consequence,
                `optimize_acqf(PosteriorStandardDeviation(gp, maximize=False))`
                actually returns -1 * minimum of the posterior standard deviation.
        """
        super().__init__(model=model, posterior_transform=posterior_transform)
        self.maximize = maximize

    @t_batch_mode_transform(expected_q=1)
    def forward(self, X: Tensor) -> Tensor:
        r"""Evaluate the posterior standard deviation on the candidate set X.

        Args:
            X: A `(b1 x ... bk) x 1 x d`-dim batched tensor of `d`-dim design points.

        Returns:
            A `(b1 x ... bk)`-dim tensor of Posterior Mean values at the
            given design points `X`.
        """
        _, std = self._mean_and_sigma(X)
        #1/2 log (2\pi\sigma^2)+0.5
        #entropy = -1*0.5*torch.log(2*math.pi*std**2) + 0.001
        #entropy = 0.5*torch.log(std*math.sqrt(2*math.pi*math.exp(1)) ) + 0.0001
        entropy = 0.5*torch.log(2*math.pi*std**2) + 0.5
        return entropy if self.maximize else -1*entropy

# ----------------Signal to noise ratio AF ------------------------------
class SignalToNoiseRatio(AnalyticAcquisitionFunction):
    def __init__(
        self,
        model: Model,
        posterior_transform: Optional[PosteriorTransform] = None,
        jitter=1e-6, max_tries=10,
        maximize: bool = True,
    ) -> None:
        r"""Single-outcome Posterior Mean.

        Args:
            model: A fitted single-outcome GP model (must be in batch mode if
                candidate sets X will be)
            posterior_transform: A PosteriorTransform. If using a multi-output model,
                a PosteriorTransform that transforms the multi-output posterior into a
                single-output posterior is required.
            maximize: If True, consider the problem a maximization problem. Note
                that if `maximize=False`, the posterior standard deviation is negated.
                As a consequence,
                `optimize_acqf(PosteriorStandardDeviation(gp, maximize=False))`
                actually returns -1 * minimum of the posterior standard deviation.
        """
        super().__init__(model=model, posterior_transform=posterior_transform)
        self.jitter   = jitter
        self.max_tries = max_tries
        self.maximize = maximize

    @t_batch_mode_transform(expected_q=1)
    def forward(self, X: Tensor) -> Tensor:
        r"""Evaluate the posterior standard deviation on the candidate set X.

        Args:
            X: A `(b1 x ... bk) x 1 x d`-dim batched tensor of `d`-dim design points.

        Returns:
            A `(b1 x ... bk)`-dim tensor of Posterior Mean values at the
            given design points `X`.
        """
        mu, std = self._mean_and_sigma(X)
        #print(mu,std)
        ratio = mu/std + 0.001
        ratio = (2*1.96*std)/mu + 0.01
        ratio = abs(std/mu) + 0.01

        return ratio if self.maximize else -1*ratio
        """
        #self.to(device=X.device)  # ensures buffers / parameters are on the same device
        posterior = self.model.posterior(
            X=X, posterior_transform=self.posterior_transform
        )
        mu = posterior.mean
        sigma = posterior.variance.sqrt().clamp_min(1e-9)  # Avoid division by zero
        ratio = mu / sigma
        return ratio if self.maximize else -ratio
        """

    def _make_pd(self, cov: torch.Tensor) -> torch.Tensor:
    # symmetrize
      cov = 0.5 * (cov + cov.transpose(-1, -2))
      jit = self.jitter
      eye   = torch.eye(cov.size(-1), device=cov.device)
      for _ in range(self.max_tries):
          try:
              return torch.linalg.cholesky(cov + jit * eye)
          except RuntimeError:
              jit *= 10
      raise RuntimeError(f"Could not PD‐fix cov with jitter up to {jit:e}")

    def posterior(self, X: torch.Tensor, **kwargs) -> MultivariateNormal:
      # get the "raw" GP posterior
      post = super().posterior(X, **kwargs)
      L    = self._make_pd(post.covariance_matrix)
      # build a new (safe) MultivariateNormal
      return MultivariateNormal(post.mean, scale_tril=L)
# ----------------Signal to noise ratio+Entropy AF ------------------------------
class EntropySNR(AnalyticAcquisitionFunction):
    def __init__(
        self,
        model: Model,
        posterior_transform: Optional[PosteriorTransform] = None,
        maximize: bool = True,
    ) -> None:
        r"""Single-outcome Posterior Mean.

        Args:
            model: A fitted single-outcome GP model (must be in batch mode if
                candidate sets X will be)
            posterior_transform: A PosteriorTransform. If using a multi-output model,
                a PosteriorTransform that transforms the multi-output posterior into a
                single-output posterior is required.
            maximize: If True, consider the problem a maximization problem. Note
                that if `maximize=False`, the posterior standard deviation is negated.
                As a consequence,
                `optimize_acqf(PosteriorStandardDeviation(gp, maximize=False))`
                actually returns -1 * minimum of the posterior standard deviation.
        """
        super().__init__(model=model, posterior_transform=posterior_transform)
        self.maximize = maximize

    @t_batch_mode_transform(expected_q=1)
    def forward(self, X: Tensor) -> Tensor:
        r"""Evaluate the posterior standard deviation on the candidate set X.

        Args:
            X: A `(b1 x ... bk) x 1 x d`-dim batched tensor of `d`-dim design points.

        Returns:
            A `(b1 x ... bk)`-dim tensor of Posterior Mean values at the
            given design points `X`.
        """
        mu, std = self._mean_and_sigma(X)
        #ratio = 0.5*torch.log(2*math.pi*std**2) - mu/std + 0.5
        ratio = 0.5*torch.log(std*math.sqrt(2*math.pi*math.exp(1)) ) + 0.5*std/mu
        return ratio if self.maximize else -ratio

In [ ]:
from sklearn import datasets, linear_model
from sklearn import datasets, linear_model

start = 0
# was 2020
num = 1000
loc = 1

X = []
for j in range(5):
  for k in range(5):
    for i in range(start,num):
      X.append([1,j,k])

train_Y_LR_total = [[] for i in range(len(map_df['NAME'].tolist()))]
reg_array = [[] for i in range(len(map_df['NAME'].tolist()))]

for loc in range(len(map_df['NAME'].tolist())):
  train_Y_LR_total[loc] = np.array([df[0][map_df['NAME'].loc[loc]].tolist()[start:num]
                                    + df[1][map_df['NAME'].loc[loc]].tolist()[start:num]
                                    + df[2][map_df['NAME'].loc[loc]].tolist()[start:num]
                                    + df[3][map_df['NAME'].loc[loc]].tolist()[start:num]
                                    + df[4][map_df['NAME'].loc[loc]].tolist()[start:num]
                                    + df[5][map_df['NAME'].loc[loc]].tolist()[start:num]
                                    + df[6][map_df['NAME'].loc[loc]].tolist()[start:num]
                                    + df[7][map_df['NAME'].loc[loc]].tolist()[start:num]
                                    + df[8][map_df['NAME'].loc[loc]].tolist()[start:num]
                                    + df[9][map_df['NAME'].loc[loc]].tolist()[start:num]
                                    + df[10][map_df['NAME'].loc[loc]].tolist()[start:num]
                                    + df[11][map_df['NAME'].loc[loc]].tolist()[start:num]
                                    + df[12][map_df['NAME'].loc[loc]].tolist()[start:num]
                                    + df[13][map_df['NAME'].loc[loc]].tolist()[start:num]
                                    + df[14][map_df['NAME'].loc[loc]].tolist()[start:num]
                                    + df[15][map_df['NAME'].loc[loc]].tolist()[start:num]
                                    + df[16][map_df['NAME'].loc[loc]].tolist()[start:num]
                                    + df[17][map_df['NAME'].loc[loc]].tolist()[start:num]
                                    + df[18][map_df['NAME'].loc[loc]].tolist()[start:num]
                                    + df[19][map_df['NAME'].loc[loc]].tolist()[start:num]
                                    + df[20][map_df['NAME'].loc[loc]].tolist()[start:num]
                                    + df[21][map_df['NAME'].loc[loc]].tolist()[start:num]
                                    + df[22][map_df['NAME'].loc[loc]].tolist()[start:num]
                                    + df[23][map_df['NAME'].loc[loc]].tolist()[start:num]
                                    + df[24][map_df['NAME'].loc[loc]].tolist()[start:num]

                                    ]).T
  reg_array[loc] = linear_model.LinearRegression()
  #print(np.shape(X), np.shape(train_Y_LR) )
  # Train the model using the training sets
  reg_array[loc].fit(X, train_Y_LR_total[loc])

  #y_pred = regr.predict(X)
  print(reg_array[loc].intercept_,reg_array[loc].coef_)
  #print(y_pred)

In [ ]:

# start_ls = 1000
# end = 2000
start_ls = 500
end = 1000
df_avg = [[] for i in range(25)]

for j in range(5):
  for k in range(5):
    for i in range(len(map_df)):
      #print(j*5+k,i, np.shape(df_avg[0]))
      df_avg[j*5+k].append(np.array(df[j*5+k][map_df['NAME'].loc[i]].tolist()[start_ls:end] ).mean() )

In [ ]:

df_avg_all = []
for i in range(len(map_df)):
  temp = []
  for j in range(25):
    temp.append(df_avg[j][i])

  df_avg_all.append(np.array(temp).mean())
  #df_avg_all.append(np.array(df_avg[i]).mean())

lr1_true = np.array([reg_array[i].intercept_[0] for i in range(len(map_df))])
lr2_true = np.array([reg_array[i].coef_[0][1] for i in range(len(map_df))])
lr3_true = np.array([reg_array[i].coef_[0][2] for i in range(len(map_df))])
#lr4 = np.array([reg_array[i].coef_[0][3] for i in range(len(map_df))])
#lr5 = np.array([reg_array[i].coef_[0][4] for i in range(len(map_df))])
#lr6 = np.array([reg_array[i].coef_[0][5] for i in range(len(map_df))])

error = []
total_value = len(map_df)
pop_100 = [100000 for i in range(total_value)]
error = []
val = []; val2 = []
for i in range(5):
  for j in range(5):
    #error.append( (df_avg[i*5+j]-(lr1+lr2*i+lr3*j) )**2 )
    val.append(sum((df_avg[i*5+j] - (lr1_true+lr2_true*i+lr3_true*j) )**2)/total_value)
    val2.append(sum(( (df_avg[i*5+j]/np.array(pop))*np.array(pop_100) - (np.array(lr1_true+lr2_true*i+lr3_true*j)/np.array(pop))*np.array(pop_100) )**2)/total_value)


sum(val)/25, sum(val2)/25
#get mean over all counties and all TCs

In [ ]:
bound0 = [-104,45.9,30,0,5]
bound1 = [-96.3,48.9,100,110,300]
def find_widest_ci_conditions(samples, num_levels1, num_levels2):
    """
    Given posterior samples of shape (S, n_counties, 3) where each draw is
    [intercept, beta_level1, beta_level2], and the number of levels for each
    factor, compute for each county which treatment‐condition index has the
    widest 95% credible interval of the predicted outcome.

    Parameters
    ----------
    samples : array‐like, shape (S, n_counties, 3)
        Posterior draws for intercept and two coefficients.
    num_levels1 : int
        Number of discrete levels for the first factor (e.g., naloxone).
    num_levels2 : int
        Number of discrete levels for the second factor (e.g., buprenorphine).

    Returns
    -------
    widest_idx : ndarray, shape (n_counties,)
        For each county, the integer index (0 to num_levels1*num_levels2−1)
        of the treatment condition with the largest 95% CI width.
    """
    arr = np.array(samples)  # shape (S, n_counties, 3)
    S, n_counties, _ = arr.shape

    # Build list of all treatment conditions (level1, level2)
    levels = [
        (l1, l2)
        for l1 in range(1, num_levels1 + 1)
        for l2 in range(1, num_levels2 + 1)
    ]
    C = len(levels)

    # Compute predicted outcomes for each draw, county, and condition
    y = np.zeros((S, n_counties, C))
    for c, (l1, l2) in enumerate(levels):
        y[:, :, c] = arr[:, :, 0] + arr[:, :, 1] * l1 + arr[:, :, 2] * l2

    # 95% credible intervals
    lower = np.percentile(y, 2.5, axis=0)   # shape (n_counties, C)
    upper = np.percentile(y, 97.5, axis=0)  # shape (n_counties, C)

    # Widths and argmax
    widths = upper - lower                  # shape (n_counties, C)
    widest_idx = np.argmax(widths, axis=1)  # shape (n_counties,)

    return widest_idx



# Running GPR-RF

In this section, we implement the two-step metamodel: a multi-output Gaussian Process Regression (MO–GPR) with response function (RF). The MO-GPR serves as a surrogate for the expensive simulation model, capturing how epidemic outcomes respond to treatment allocations across counties.

Let each county be indexed by
$c \in \mathcal{C}$
and each treatment condition by
$(n,b) \in \mathcal{T}$,
where $n$ and $b$ represent naloxone and buprenorphine intervention levels, respectively.

We denote the simulation output for county $c$ under treatment $(n,b)$ at design iteration $t$ as
$
y.
$

The dataset at iteration $t$ is:
$
\mathcal{D}_t = \mathcal{D}_{t-1} \cup \{ (\mathbf{x}_c, (c, n, b), y) \},
$
where $\mathbf{x}_c$ includes spatial features (county centroid) and contextual features (population, overdose trends, dispensing rates).

The MO–GPR posterior for a new input $x_c^*$ is:
\[
$\boldsymbol{\mu}(\mathbf{x}_c^*)$ =
\begin{bmatrix}
\mu_0(\mathbf{x}_c^*),
\mu_n(\mathbf{x}_c^*),
\mu_b(\mathbf{x}_c^*)
\end{bmatrix},
$\boldsymbol{K}(\mathbf{x}_c^*)$
\]

Here, $\mu_0, \mu_n, \mu_b$ are regression coefficients approximating baseline mortality and treatment effects. Their uncertainty, encoded in $K$, is estimated from replicate simulation runs.

This section of code therefore:
1. Defines the kernel.
2. Fits the GPR to current simulation data.
3. Samples posterior mean and variance for each county-treatment combination.
4. Updates the metamodel to guide the sequential design.

In [ ]:
from torch.optim import SGD, Adam, LBFGS, RMSprop, Adagrad
from botorch.utils.transforms import normalize, unnormalize
from botorch.models.transforms.input import Normalize
from botorch.optim.initializers import initialize_q_batch_nonneg
#from botorch.optim.fit import fit_gpytorch_torch

import copy
import warnings


warnings.filterwarnings("ignore")
torch.set_warn_always(True)

step = 10
batch_size = 5
hetero_gpr = False

batch_size2 = batch_size+step
loc = 1
normalize_flag = False
################################################################################################
X_levels = []

for j in range(5):
  for k in range(5):
    for i in range(batch_size):
      X_levels.append([1,j,k])

train_gp_LR = [[] for i in range(len(map_df['NAME'].tolist()))]
reg_array_gp = [[] for i in range(len(map_df['NAME'].tolist()))]

for loc in range(len(map_df['NAME'].tolist())):
  train_gp_LR[loc] = np.array([df[0][map_df['NAME'].loc[loc]].tolist()[:batch_size]
                               + df[1][map_df['NAME'].loc[loc]].tolist()[:batch_size]
                               + df[2][map_df['NAME'].loc[loc]].tolist()[:batch_size]
                               + df[3][map_df['NAME'].loc[loc]].tolist()[:batch_size]
                               + df[4][map_df['NAME'].loc[loc]].tolist()[:batch_size]
                              + df[5][map_df['NAME'].loc[loc]].tolist()[:batch_size]
                              + df[6][map_df['NAME'].loc[loc]].tolist()[:batch_size]
                              + df[7][map_df['NAME'].loc[loc]].tolist()[:batch_size]
                              + df[8][map_df['NAME'].loc[loc]].tolist()[:batch_size]
                              + df[9][map_df['NAME'].loc[loc]].tolist()[:batch_size]
                              + df[10][map_df['NAME'].loc[loc]].tolist()[:batch_size]
                              + df[11][map_df['NAME'].loc[loc]].tolist()[:batch_size]
                              + df[12][map_df['NAME'].loc[loc]].tolist()[:batch_size]
                              + df[13][map_df['NAME'].loc[loc]].tolist()[:batch_size]
                              + df[14][map_df['NAME'].loc[loc]].tolist()[:batch_size]
                              + df[15][map_df['NAME'].loc[loc]].tolist()[:batch_size]
                              + df[16][map_df['NAME'].loc[loc]].tolist()[:batch_size]
                              + df[17][map_df['NAME'].loc[loc]].tolist()[:batch_size]
                              + df[18][map_df['NAME'].loc[loc]].tolist()[:batch_size]
                              + df[19][map_df['NAME'].loc[loc]].tolist()[:batch_size]
                              + df[20][map_df['NAME'].loc[loc]].tolist()[:batch_size]
                              + df[21][map_df['NAME'].loc[loc]].tolist()[:batch_size]
                              + df[22][map_df['NAME'].loc[loc]].tolist()[:batch_size]
                              + df[23][map_df['NAME'].loc[loc]].tolist()[:batch_size]
                              + df[24][map_df['NAME'].loc[loc]].tolist()[:batch_size]
                               ]).T

  reg_array_gp[loc] = linear_model.LinearRegression()
  # Train the model using the training sets
  reg_array_gp[loc].fit(X_levels, train_gp_LR[loc])


train_reg1 = torch.Tensor([round(reg_array_gp[i].intercept_[0],3) for i in range(len(map_df['NAME'].tolist()))])
train_reg2 = torch.Tensor([round(reg_array_gp[i].coef_[0][1],3) for i in range(len(map_df['NAME'].tolist()))])
train_reg3 = torch.Tensor([round(reg_array_gp[i].coef_[0][2],3) for i in range(len(map_df['NAME'].tolist()))])

#train_reg4 = torch.Tensor([round(reg_array_gp[i].coef_[0][3],3) for i in range(len(map_df['NAME'].tolist()))])
#train_reg5 = torch.Tensor([round(reg_array_gp[i].coef_[0][4],3) for i in range(len(map_df['NAME'].tolist()))])

test_reg1 = torch.Tensor([round(reg_array[i].intercept_[0],3) for i in range(len(map_df['NAME'].tolist()))])
test_reg2 = torch.Tensor([round(reg_array[i].coef_[0][1],3) for i in range(len(map_df['NAME'].tolist()))])
test_reg3 = torch.Tensor([round(reg_array[i].coef_[0][2],3) for i in range(len(map_df['NAME'].tolist()))])
#test_reg4 = torch.Tensor([round(reg_array[i].coef_[0][3],3) for i in range(len(map_df['NAME'].tolist()))])
#test_reg5 = torch.Tensor([round(reg_array[i].coef_[0][4],3) for i in range(len(map_df['NAME'].tolist()))])

par1 = [reg_array[i].intercept_[0] for i in range(len(map_df['NAME'].tolist()))]
par2 = [reg_array[i].coef_[0][1] for i in range(len(map_df['NAME'].tolist()))]
par3 = [reg_array[i].coef_[0][2] for i in range(len(map_df['NAME'].tolist()))]
#par4 = [reg_array[i].coef_[0][3] for i in range(len(map_df['NAME'].tolist()))]
#par5 = [reg_array[i].coef_[0][4] for i in range(len(map_df['NAME'].tolist()))]

################################################################################################
true_intercepts = 2.0 + np.random.rand() * 0.002 * train_X.sum(dim=1)
true_slopes = 3.0 + np.random.rand() * 0.001 * train_X.sum(dim=1)
count1 = [0 for i in range(len(map_df))]
count2 = [0 for i in range(len(map_df))]
count3 = [0 for i in range(len(map_df))]
count4 = [0 for i in range(len(map_df))]

if normalize_flag:
  bounds1 = [train_X.min(), train_X.max()]
  bounds2 = [train_reg1.min(), train_reg1.max()]
  bounds3 = [train_reg2.min(), train_reg2.max()]
  train_X = normalize(train_X, bounds1)
  #train_Y = normalize(train_Y, bounds2)
  train_reg1 = normalize(train_reg1, bounds2)
  train_reg2 = normalize(train_reg2, bounds3)

num_par = 3
ls_y = [[round(train_reg1[i].tolist(),3), round(train_reg2[i].tolist(),3), round(train_reg3[i].tolist(),3),
         #round(train_reg4[i].tolist(),3), round(train_reg5[i].tolist(),3)
         ]
        for i in range(len(map_df))]
ls_y_test = [[round(test_reg1[i].tolist(),3), round(test_reg2[i].tolist(),3), round(test_reg3[i].tolist(),3),
         #round(test_reg4[i].tolist(),3), round(test_reg5[i].tolist(),3)
         ]
        for i in range(len(map_df))]
train_Y = torch.tensor(ls_y, dtype=torch.float64)
test_Y = torch.tensor(ls_y_test, dtype=torch.float64)

train_X1 = train_X
test_X = copy.deepcopy(train_X)
train_X2 = train_X
train_X3 = train_X
train_X4 = train_X
train_X5 = train_X
train_Y1 = train_Y
train_Y2 = train_Y
train_Y3 = train_Y

compare = 2
covar_module_comp = [0 for i in range(compare)]
comp_flag = False

train_reg5 = train_reg1

train_gp_LR1 = train_gp_LR
train_gp_LR2 = train_gp_LR
train_gp_LR3 = train_gp_LR
train_gp_LR4 = train_gp_LR
train_gp_LR5 = train_gp_LR
reg_array_gp1 = reg_array_gp
reg_array_gp2 = reg_array_gp
reg_array_gp3 = reg_array_gp
reg_array_gp4 = reg_array_gp
reg_array_gp5 = reg_array_gp

mse_all = []
mse_all2 = []
mse_all3 = []
mse_all_comp = [[] for i in range(compare)]
mse_all_comp2 = [[] for i in range(compare)]
gp_value = []
gp_value2 = []
gp_rate = []
gp_rate2 = []
gp_rate3 = []
gp_rate4 = []
adams_mse_rate = []
adams_mse_count = []
bge_all = []
bge_all2 = []
sample_complexity = []
sample_complexity2 = []
samples = [0]*len(map_df)

flag = False

entropy_flag = False

num_samples = [list(i.flatten()) for i in train_gp_LR1]
sum_num_samples = sum(xyz, [])
print("shape of train set:", np.shape(sum_num_samples))
#train_val1 = train_reg.detach().numpy().flatten()
#train_val2 = train_Y2.detach().numpy().flatten()

N_BATCH = 150 #int(20000/step) #was 2000

covar_module = (ScaleKernel(RBFKernel(active_dims=[0,1]))+(RBFKernel(active_dims=[2])+RBFKernel(active_dims=[3]))*PeriodicKernel())

covar_module = RBFKernel(active_dims=[0,1])+RBFKernel(active_dims=[2])#*PeriodicKernel()

if hetero_gpr:
  X = train_X1.double()                     # [N, d]
  y0 = train_Y[:, 0:1].double()             # [N, 1]
  y1 = train_Y[:, 1:2].double()             # [N, 1]
  y2 = train_Y[:, 2:2+1].double()           # [N, 1]
  v0 = train_Yvar[:, 0:1].double() *scale          # [N, 1]
  v1 = train_Yvar[:, 1:2].double() *scale         # [N, 1]
  v2 = train_Yvar[:, 2:3].double() *scale         # [N, 1]

  covar0 = deepcopy(covar_module)
  covar1 = deepcopy(covar_module)
  covar2 = deepcopy(covar_module)

  m0 = SingleTaskGP(X, y0, v0, covar_module=covar0).double()
  m1 = SingleTaskGP(X, y1, v1, covar_module=covar1).double()
  m2 = SingleTaskGP(X, y2, v2, covar_module=covar2).double()
  gp  = ModelListGP(m0, m1, m2).double()
  gp_mll = SumMarginalLogLikelihood(gp.likelihood, gp).double()
else:
  gp = SingleTaskGP(train_X1, train_Y.double(),
                    #outcome_transform=Standardize(m=num_par),
                    #input_transform=Normalize(d=train_X.shape[-1]),
                    #outcome_transform=None,
                    covar_module=covar_module
                    )

  gp_mll = ExactMarginalLogLikelihood(gp.likelihood, gp)

fit_gpytorch_mll(mll=gp_mll)

val = round(1.0/num_par,3)
ls_val = [val for i in range(num_par)]
ls_val = [0.8,0.1,0.1]
weights = torch.tensor(ls_val, dtype=torch.double)
posterior_transform = ScalarizedPosteriorTransform(weights)

SNR = SignalToNoiseRatio(gp.double(), posterior_transform=posterior_transform, jitter=1e-2, max_tries=10)

iteration = 0
round_robin = 5
stop_sample = [[batch_size]*5*5 for i in range(len(map_df))]

while iteration < N_BATCH:
  iteration += 1
  #was 0.07
  lr_val = max(0.07, 0.3 - iteration*0.0000001)
  with gpytorch.settings.cholesky_jitter(1e-4):

    training_iter = 1

    if hetero_gpr:
      gp_mll = SumMarginalLogLikelihood(gp.likelihood, gp).double()
      fit_gpytorch_mll(mll=gp_mll)
    else:
      fit_gpytorch_mll(mll=gp_mll)

    SNR = SignalToNoiseRatio(gp.double(), maximize=True, posterior_transform=posterior_transform,
                               jitter=1e-2, max_tries=10)

    C = choice.T
    C = torch.as_tensor(C, dtype=torch.float64, device=gp.train_inputs[0].device)

    for _ in range(20):                      # retry up to 5 times
      try:
          new_pnt, new_value = optimize_acqf_discrete(
          acq_function=SNR.double(),
          #bounds= bounds3,
          q=1,
          choices = C,
          )
          break
      except RuntimeError as e:
          if "cholesky" in str(e).lower():
              #gp.likelihood.noise_covar.noise.data *= 0.5 #1.1   # bump noise, retry
              with torch.no_grad():
                # Increase noise and ensure it's double precision
                gp.likelihood.noise_covar.noise.data = (
                    gp.likelihood.noise_covar.noise.data.double().clamp(min=1e-6) #* 1.5
                )
                #print("noise:",gp.likelihood.noise_covar.noise.data.double())
                # Force all model parameters to double
                for param in gp.parameters():
                    if param.dtype != torch.float64:
                        param.data = param.data.double()
          else:
              raise

  idx = get_index(new_pnt,map_df)
  if idx is None:
    print("Here in idx:", new_pnt)
    new_pnt22 = adjust_loc(new_pnt, map_df)
    print("Here in idx:", new_pnt22)
    new_pnt = new_pnt22
    #print(pointsame1, counter)
    idx = get_index(new_pnt,map_df)

  if flag or max(samples)>48000:
    print("Choosing one of the neighbors as the idx has exhausted all samples")
    state = "North Dakota"
    county = map_df['NAME'].loc[idx]
    idx = get_neighboring_counties(state, map_df, county)
    old_pnt = new_pnt
    new_pnt = torch.tensor([newls[idx]])
    print(idx, map_df['NAME'].loc[idx], old_pnt, new_pnt)
    flag = False

  loc = idx
  samples = [np.shape(train_gp_LR1[i])[0] for i in range(len(map_df))]
  #if samples[0] <= batch_size*5:
  #  idx = 0
  #  loc = idx
  loc = idx
  if iteration <= 1:
    ci_ind = 0
  else:
    ci_ind = widest_indices[loc]
  train_gp_LR1[idx] = np.append(train_gp_LR1[idx], df[ci_ind][map_df['NAME'].loc[loc]].tolist()[stop_sample[loc][ci_ind]:stop_sample[loc][ci_ind]+step]).T
  stop_sample[loc][ci_ind] = stop_sample[loc][ci_ind]+step

  X_levels1 = []
  for l1 in range(5):
    for l2 in range(5):
      len_condition = stop_sample[loc][l1*5+l2]
      for _ in range(len_condition):
        #print("inside loop ", l1,l2, l1*4+l2, len_condition)
        X_levels1.append([1,l1,l2])

  try:
    reg_array_gp1[idx].fit(X_levels1, np.array(train_gp_LR1[idx]).reshape(-1,1) )
  except:
    print("Continue for loop\n")
    print(np.shape(X_levels1), np.shape(train_gp_LR1[idx]))

  batch_size2 = batch_size2 + step


  reg_value_gp1 = round(reg_array_gp1[idx].intercept_[0],3)
  reg_value_gp12 = round(reg_array_gp1[idx].coef_[0][1],3)
  reg_value_gp13 = round(reg_array_gp1[idx].coef_[0][2],3)
  #reg_value_gp14 = round(reg_array_gp1[idx].coef_[0][3],3)
  #reg_value_gp15 = round(reg_array_gp1[idx].coef_[0][4],3)


  count1[idx] += 1
  count2[idx] += 1

  if hetero_gpr:
    yvec = np.asarray(train_gp_LR1[idx]).reshape(-1, 1)
    beta_hat, var_diag = ols_coeffs_and_var(Xmat, yvec)
    for i in range(3):
      train_Y[idx, i] = float(beta_hat[i])
      train_Yvar[idx, i] = float(var_diag[i])

    sample_sz = np.array([len(train_gp_LR1[i]) for i in range(len(map_df))])
    base_scale = 10.0
    sample_sz = np.reshape(sample_sz, [-1,1])
    sample_sz = torch.tensor(scale/sample_sz)

    X = train_X1.double()
    y_list = []
    v_list = []

    for i in range(3):
        y_list.append(train_Y[:, i:i+1].double())
        v_list.append(train_Yvar[:, i:i+1].double() * sample_sz)

    y0, y1, y2 = y_list
    v0, v1, v2 = v_list
    # Add minimum floor to prevent numerical issues
    v0 = torch.clamp(v0, min=1e-6)
    v1 = torch.clamp(v1, min=1e-6)
    v2 = torch.clamp(v2, min=1e-6)

    models = [SingleTaskGP(X, y, v, covar_module=covar_module).double()
          for y, v in [(y0, v0), (y1, v1), (y2, v2)]]

    gp = ModelListGP(*models).double()

    with gpytorch.settings.cholesky_jitter(1e-1):
      try:
          gp_mll = SumMarginalLogLikelihood(gp.likelihood, gp).double()
          fit_gpytorch_mll(gp_mll)
      except:
          print("error in second fit")

    Xquery = torch.tensor(newls, dtype=torch.double)

    # Get posteriors and extract means
    val_lr1, val_lr2, val_lr3 = [
      gp.models[i].posterior(Xquery).mean.squeeze(-1).detach().cpu().numpy()
      for i in range(3)
      ]

  else:
    train_X1 = torch.cat([train_X1,torch.tensor(new_pnt)]).double()
    train_Y = torch.cat([train_Y,torch.tensor([[reg_value_gp1,reg_value_gp12,reg_value_gp13,]])]).double()

    with gpytorch.settings.cholesky_max_tries(2000000):
      gp = SingleTaskGP(train_X1, train_Y,
                        #input_transform=Normalize(d=train_X1.shape[-1]),
                        #outcome_transform=Standardize(m=num_par),
                        #outcome_transform=None,
                        covar_module=covar_module
                        ).double()

      #gp.train()
      gp_mll = ExactMarginalLogLikelihood(gp.likelihood, gp)

      #fit_gpytorch_mll(mll=gp_mll)

      val_lr = gp.posterior(torch.Tensor(newls),).mean.tolist()
      val_lr1 = [val_lr[i][0] for i in range(len(val_lr))]
      val_lr2 = [val_lr[i][1] for i in range(len(val_lr))]
      val_lr3 = [val_lr[i][2] for i in range(len(val_lr))]
      #val_lr4 = [val_lr[i][3] for i in range(len(val_lr))]
      #val_lr5 = [val_lr[i][4] for i in range(len(val_lr))]

    total_value = len(map_df['NAME'].tolist())

    mse_val_intercept = (sum(par1 -np.array(val_lr1) )**2) /total_value
    mse_val_covariate = (sum(par2 - np.array(val_lr2) )**2) /total_value
    mse_val_covariate2 = (sum(par3 - np.array(val_lr3) )**2) /total_value
    #mse_val_covariate3 = (sum(par4 - np.array(val_lr4) )**2) /total_value

    print("\nIteration:", iteration,"/",N_BATCH)
    print(new_pnt,int(new_value),map_df['NAME'].loc[idx], reg_value_gp1,
          #new_pnt2,int(new_value2),map_df['NAME'].loc[idx2], reg_value_gp2,
          #'\n',new_pnt3,int(new_value3),map_df['NAME'].loc[idx3], reg_value_gp3,
          #new_pnt4,int(new_value4),map_df['NAME'].loc[idx4], reg_value_gp4,
          '\n',"MSE:",mse_val_intercept, mse_val_covariate, mse_val_covariate2, #mse_val_covariate3,
          '\n',gp.posterior(torch.Tensor(new_pnt)).mean.detach().numpy().flatten(),
         # gp2.posterior(torch.Tensor(new_pnt2)).mean.detach().numpy().flatten(),
         # '\n', "BGE:", bge_value, bge_value2
          )

    loc=0
    create_value = torch.Tensor([[map_df['geometry'].loc[loc].centroid.x, map_df['geometry'].loc[loc].centroid.y,
                                  #income[loc],
                                  pop_den[loc], #black_p[loc], #native_p[loc]
                                    ]])
    #create_value = torch.Tensor([[map_df['geometry'].loc[loc].centroid.x, map_df['geometry'].loc[loc].centroid.y]])
    print("Allegheny County intercept:",gp.posterior(create_value).mean.detach().numpy().flatten()
  ,#gp2.posterior(create_value).mean.detach().numpy().flatten(),
      )

    loc=11
    create_value = torch.Tensor([[map_df['geometry'].loc[loc].centroid.x, map_df['geometry'].loc[loc].centroid.y,
                                  #income[loc],
                                  pop_den[loc], #black_p[loc], #native_p[loc]
                                    ]])
    print("Cameron County intercept:",gp.posterior(create_value).mean.detach().numpy().flatten()
  ,#gp2.posterior(create_value).mean.detach().numpy().flatten(),
    )

  lr1 = np.array(val_lr1)
  lr2 = np.array(val_lr2)
  lr3 = np.array(val_lr3)
  #lr4 = np.array(val_lr4)
  #lr5 = np.array(val_lr5)
  gp_value.append(sum((df_avg[0]- lr1 )**2))
  gp_value2.append(sum((df_avg[1]- (lr1+lr2))**2 ))

  gp_rate.append(sum((df_avg[0]/np.array(pop) - lr1/np.array(pop) )**2)/total_value)

  val = []
  val2= []
  val3 = []
  pop_100 = [100000 for i in range(total_value)]
  for x in range(5):
    for y in range(5):
      val.append(sum((df_avg[x*5+y] - (lr1+lr2*x+lr3*y) )**2)/total_value)
      val2.append(sum(( (df_avg[x*5+y]/np.array(pop)*np.array(pop_100)) - (np.array(lr1+lr2*x+lr3*y)/np.array(pop)*np.array(pop_100)) )**2)/total_value)
      val3.append(sum( abs( (df_avg[x*5+y]/np.array(pop)*np.array(pop_100)) -
        (np.array(lr1+lr2*x+lr3*y)/np.array(pop)*np.array(pop_100))) /(df_avg[x*5+y]/np.array(pop)*np.array(pop_100)))/total_value)
  gp_rate2.append(sum(val)/25)
  gp_rate3.append(sum(val2)/25)
  gp_rate4.append(sum(val3)/25)

  num_samples = [list(i.flatten()) for i in train_gp_LR1]
  ################################################################################################
  sample_post = []
  for _ in range(100):
    sample_post.append(gp.posterior(torch.Tensor(newls)).sample().tolist()[0])

  samples_post = np.array(sample_post)
  widest_indices = find_widest_ci_conditions(samples_post, num_levels1=5, num_levels2=5)
  ################################################################################################
  sum_num_samples = sum(num_samples, [])
  print(np.shape(sum_num_samples), np.shape(X_levels1), np.shape(train_gp_LR1[idx]), gp_rate2[iteration-1],
        gp_rate3[iteration-1], gp_rate4[iteration-1])
  sample_complexity.append(np.shape(sum_num_samples)[0])

## Analysis

In [ ]:
df_true = pd.DataFrame(columns=['Name','intercept','coef1','coef2','LR MSE rate','GPR MSE rate','GPR MSE count'])
df_true['Name'] = map_df['NAME'].tolist()
df_true['intercept'] = (lr1_true-lr1)**2
df_true['coef1'] = (lr2_true-lr2)**2
df_true['coef2'] = (lr3_true-lr3)**2

lr_true = []
gpr_rate = []
gpr_count = []
#for k in range(len(map_df)):
val = []
val2 = []
val3 = []
for i in range(5):
  for j in range(5):
    val.append( (df[i*5+j].mean().tolist()/np.array(pop)*np.array(pop_100)- ((lr1_true+lr2_true*i+lr3_true*j)/np.array(pop)*np.array(pop_100) ) )**2 )
    val2.append( ((df[i*5+j].mean().tolist()/np.array(pop)*np.array(pop_100))- ((lr1+lr2*i+lr3*j)/np.array(pop)*np.array(pop_100)) )**2 )
    val3.append( (df[i*5+j].mean().tolist() - (lr1+lr2*i+lr3*j) )**2 )


  #lr_true.append(sum(val)/25)
  #gpr_rate.append(sum(val2)/25)
  #gpr_count.append(sum(val3)/25)
df_true['LR MSE rate'] = np.average(val,axis=0)
df_true['GPR MSE rate'] = np.average(val2,axis=0)
df_true['GPR MSE count'] = np.average(val3,axis=0)
df_true['mean death rate'] = df_avg_all/np.array(pop)*np.array(pop_100)
df_true['mean death count'] = df_avg_all
df_true.to_csv('df_true.csv')

## Compare

In [ ]:
compare = pd.DataFrame(data=zip(sample_complexity,gp_rate3, gp_rate4), columns=['sample','mse','rel_error'])
compare.to_csv("pa25_seqdes.csv", index=False)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# ---------- file paths ----------
#file_rand = "pa25_2.csv"          # one-stage sequential design
file_rand = "pa25_seqdes.csv"
file_seq  = "pa25_seqdes2.csv"   # two-stage sequential design

# ---------- read & transform ----------
rand_df = pd.read_csv(file_rand)
seq_df  = pd.read_csv(file_seq)

for df_plot in (rand_df, seq_df):
    df_plot["rel_error_pct"] = df_plot["rel_error"] * 100.0

plt.style.use("ggplot")
m=60
# --------------------------------------------------------------------
# 1) MSE plot
# --------------------------------------------------------------------
plt.figure(figsize=(6, 4))
plt.plot(rand_df["sample"].loc[:m], rand_df["mse"].loc[:m], label="one-stage sequential design")
plt.plot(seq_df["sample"],  seq_df["mse"],  label="two-stage sequential design")
plt.xlabel("Number of samples")
plt.ylabel("MSE")
plt.title("Mean-squared error vs. sample size")
plt.legend()
plt.tight_layout()
plt.show()

# --------------------------------------------------------------------
# 2) Relative-error plot
# --------------------------------------------------------------------
plt.figure(figsize=(6, 4))
plt.plot(rand_df["sample"].loc[:m], rand_df["rel_error_pct"].loc[:m], label="one-stage sequential design")
plt.plot(seq_df["sample"],  seq_df["rel_error_pct"],  label="two-stage sequential design")
plt.xlabel("Number of samples")
plt.ylabel("Relative error (%)")
#plt.title("Relative error vs. sample size")
plt.legend()
plt.tight_layout()
plt.show()